---
Lógica Computacional TP1.Exercício1
======================
##### G01 - Bruno Dias da Gião A96544, João Luis da Cruz Pereira A95375, David Alberto Agra A95726


---

## Exercício 1 - Enunciado


1. Pretende-se construir o horário semanal de aulas de uma turma.
    1. Existe um conjunto de salas $S$ classificadas em “grandes” e “pequenas”.
    2. O tempo do horário está organizado em “slots” de uma hora. O total do tempo disponível é 5 horas de manhã e 5 horas às tarde.
    3. Existe um conjunto $D\,$ de disciplinas.  Cada disciplina tem um atributo $d\in\{1,2\}$ que classifica a duração de cada sessão (um ou dois “slots”) , um atributo $a\in\{2,3\}$ que define o número de sessões semanais e um atributo $\,s\in\{0,1\}\,$ que diz se a sessão necessita de uma sala grande ou não.
     d. Existe um conjunto $P$ de professores. Cada professor tem associado um conjunto  $h$ das disciplinas que está habilitado a lecionar.
     e. O horário está organizado em sessões concorrentes onde cada sessão é definido por uma disciplina desce que salas e professores verifiquem as seguintes restrições.
        1. Para cada disciplina todas as aulas decorrem na mesma sala e com o mesmo professor.
        2. O número total de horas lecionadas por cada professor está num intervalo de $\pm 20\%\,$do número médio de horas lecionadas pela totalidade dos professores.
        3. Nenhuma sala pode ser ocupada simultaneamente por mais do que uma aula e nenhum professor pode lecionar simultaneamente mais do que uma aula.
        4. Em cada disciplina, cada aula é lecionada por um professor habilitado para essa disciplina e ocorre numa sala de tamanho apropriado à disciplina.
        


    Use o package `ortools` para encontrar uma solução que verifique todas as restrições e maximize o número de partes de dia (manhãs ou tardes) que estão livres de qualquer aula.
---

## Exercício 1 - Solução

### Exercício 1 - Testes

Variáveis de testes (correr antes das restrições)

In [243]:
S = 3
D = 3
P = 3

gp = [0, 1, 1] # salas x grandes pequenas
ph = [[0, 1, 0], [1, 1, 0], [0, 0, 1]] # professores x disciplinas
dd = [1, 2, 1] # disciplinas x duração
na = [2, 3, 2] # diciplinas x nº aulas
ts = [0, 1, 0] # diciplinas x tamanho sala

### Exercício 1 - Código

In [244]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')

In [245]:
dias = 1
horas_manha = 5
horas_tarde = 5
h = {}

In [246]:
for dia in range(dias):
    h[dia] = {}
    for hora in range(horas_manha+horas_tarde):
        h[dia][hora] = {}
        for s in range(S):
            h[dia][hora][s] = {}
            for d in range(D):
                h[dia][hora][s][d] = {}
                for p in range(P):
                    h[dia][hora][s][d][p] = solver.BoolVar("[%i][%i][%i][%i][%i]" % (dia, hora, s, d, p))

# Professor poder dar a disciplina
for dia in range(dias):
    for hora in range(horas_manha+horas_tarde):
        for s in range(S):
            for d in range(D):
                for p in range(P):
                    solver.Add(h[dia][hora][s][d][p] <= ph[p][d])

# Sala tem o tamanho certo
#for dia in range(dias):
#    for hora in range(horas_manha+horas_tarde):
#        for s in range(S):
#            for d in range(D):
#                solver.Add(h[dia][hora][s][d][p] == gp[s] * ts[d])
                    
# Número de aulas correto
for d in range(D):
    solver.Add(sum(h[dia][hora][s][d][p] for dia in range(dias) for hora in range(horas_manha+horas_tarde) for s in range(S) for p in range(P)) == na[d])

# Duração de aula correta
#for dia in range(dias):
#    for hora in range(horas_manha + horas_tarde):
#        for s in range(S):
#            for p in range(P):
#                for d in range(D):
#                    if dd[d] == 2 and hora < horas_manha + horas_tarde - 1:
#                       solver.Add(h[dia][hora][s][d][p] == 1)
#                       solver.Add(h[dia][hora + 1][s][d][p] == 1)
#                    elif dd[d] == 1:
#                        solver.Add(h[dia][hora][s][d][p] == 1)
#                        if hora < horas_manha + horas_tarde - 1:
#                            solver.Add(h[dia][hora + 1][s][d][p] == 0)


# Uma aula por hora por professor
for dia in range(dias):
    for hora in range(horas_manha+horas_tarde):
        for p in range(P):
            solver.Add(sum(h[dia][hora][s][d][p] for d in range(D) for s in range(S)) <= 1)

# A sala não pode estar ocupada por duas aulas ao mesmo tempo
for dia in range(dias):
    for hora in range(horas_manha+horas_tarde):
        for s in range(S):
            solver.Add(sum(h[dia][hora][s][d][p] for d in range(D) for p in range(P)) <= 1)
        
# Um professor por disciplina
for d in range(D):
    solver.Add(sum(h[dia][hora][s][d][p] for dia in range(dias) for hora in range(horas_manha + horas_tarde) for s in range(S) for p in range(P)) <= 1)

# A mesma sala para cada disciplina
for s in range(S):
    for d in range(D):
        solver.Add(sum(h[dia][hora][s][d][p] for dia in range(dias) for hora in range(horas_manha+horas_tarde) for p in range(P)) <= 1)


In [247]:
def print_horario():
    # Create a 5D list to store the schedule
    schedule = [[[[[0 for _ in range(P)] for _ in range(D)] for _ in range(S)] for _ in range(horas_manha + horas_tarde)] for _ in range(dias)]

    # Extract the solution from the solver and populate the schedule
    for dia in range(dias):
        for hora in range(horas_manha + horas_tarde):
            for s in range(S):
                for d in range(D):
                    for p in range(P):
                        value = h[dia][hora][s][d][p].solution_value()
                        if value == 1:
                            schedule[dia][hora][s][d][p] = value

    # Print the schedule in a human-readable format
    for dia in range(dias):
        print(f"Day {dia + 1}:")
        for hora in range(horas_manha + horas_tarde):
            print(f"Hour {hora + 1}:", end=" ")
            for s in range(S):
                for d in range(D):
                    for p in range(P):
                        if h[dia][hora][s][d][p].solution_value() == 1:
                            print(f"Room {s + 1}, Discipline {d + 1}, Professor {p + 1}")
            print("\n")
        print()  # Separate days with an empty line

In [248]:
r = solver.Solve()

if r == pywraplp.Solver.OPTIMAL:
    print("Solução encontrada") 
    print_horario()
else:
    print("Não foi encontrada solução")

Não foi encontrada solução
